<a href="https://colab.research.google.com/github/ATalaveraDev/social-network-extraction-info/blob/main/ERS_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENUNCIADO
Ejercicio de twitter. Encontrar los 5 estados más felices de USA de acuerdo al análisis de sentimiento con AFFINN. 

Contenido. un fichero .ipynb con enunciado. Si no se hace completo indicad en un comentario lo que se ha conseguido Mostrar mapa USA con los estados coloreados por felicidad

Realizar un mapa de España que permita ver dónde se tuitea y en qué idioma se tuitea

## PARTE 1:
Encontrar los 5 estados más felices de USA de acuerdo al análisis de sentimiento con AFFINN 

####Dependencias

In [ ]:
!pip install tweepy

####Obtener los Estados de Estados Unidos

En esta parte he hecho scrapping de una web del google developers portal que contiene el codigo de cada estado, su nombre y sus coordenadas. El motivo de esto es porque de cada Tweet voy a recoger el valor del atributo "location" el cual presenta el formato: "nombre_estado, CODIGO_ESTADO"

In [ ]:
import requests
from bs4 import BeautifulSoup

URL = "https://developers.google.com/public-data/docs/canonical/states_csv"
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

states = []
table = soup.select_one('table')
for row in table.select('tr'):
  tds = row.find_all('td')
  if (len(tds) > 0):
    cells = [d.text for d in tds]
    states.append({ 'state': cells[0], 'name': cells[3], 'latitude': cells[1], 'longitude': cells[2] })

# DEBUGGING del resultado
#states

####OAuth authentication
Autenticación en la API de Twitter mediante OAuth

In [1]:
import tweepy

auth = tweepy.OAuthHandler('GJdVKUjoEl31WLHlgIRYxVmYr', 'HVaOvQWoHIkiHZh2yPNb0YMxwR6waHagtoR4hcKURo6nhvGxwL')
auth.set_access_token('1254452545583529991-g0sI2XhrtijaQ5zA3NNPRwCfuSKrCr', '4GLEprsRqsKlHPelKC4YirECbHTQYsJJGYiy0JN2OuRiH')

api = tweepy.API(auth)

####Tweets Selection

En esta sección se obtienen los Tweets a través de la funcionalidad de streaming de la API. Por defecto el tiempo de streaming es de 1 minuto pero se puede configurar, para ello basta con editar la linea "time.sleep(60)"

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
import time

class StdOutListener(StreamListener):
    def on_data(self, data):
      tweet = json.loads(data)
      tweets.append({ 'text': tweet['text'], 'place': tweet['place']['full_name'] })
      return True

    def on_error(self, status):
        print(status)

tweets = []
l = StdOutListener()
stream = Stream(auth, l)
stream.filter(languages=['en'], locations=[-171.791110603, 18.91619, -66.96466, 71.3577635769], is_async=True)
time.sleep(30)
stream.disconnect()


###Análisis de Sentimiento

Es necesario importar al proyecto el archivo AFINN-111.txt. Tras ello he configurado el programa para que haga un parser del documento y así crear un dictionary de sentimientos que será utilizado posteriormente para asignar a cada Tweet un "índice de felicidad"

In [ ]:
feelings = open('AFINN-111.txt', 'r')

def initFeelingsDictionary():
  dict = {}
  for feeling in feelings:
    feelingSplit = feeling.split('\\n')
    feel = feelingSplit[0].replace('\n', '').split('\t')
    dict[feel[0]] = feel[1]
  return dict

affinn = initFeelingsDictionary()

# Para evitar problemas con los streamings recogí algunos tweets y los utilice como datos. De esta forma no habia necesidad de utilizar 
# los streams en todas las fases del desarrollo
#testData = [{'text': '@barbiebayxx @NaijaEverything We from Africa it ain’t née plus I like younger girls 🌚🌚 she looks sweet', 'place': 'Georgia, USA'}, {'text': '@lonliestwolf Gooood....goooood https://t.co/QdQKaWl8xK', 'place': 'Columbus, OH'}, {'text': 'it’d be a lie to say that shit didn’t hurt my feelings.', 'place': 'Dallas, TX'}, {'text': 'I can’t complain...🤷🏾\u200d♂️... it was all 4 the taking 🙇🏾\u200d♂️', 'place': 'Oakland, CA'}, {'text': 'Flat Rock / Coon Creek: Running https://t.co/XnDGONy515', 'place': 'Alabama, USA'}, {'text': 'Don’t let No one get comfortable Disrespecting You!!!', 'place': 'Scio, MI'}, {'text': '@Thomas1774Paine Just like he was a Russian puppet right Eric? \nHow’s that Russian Collusion fairytale going? Have… https://t.co/38iy7V6MgC', 'place': 'Russellton, PA'}, {'text': '@Oluomachi_u I believe in you!😭', 'place': 'Chicago, IL'}, {'text': '@iTS_MeMadiO You work at the 🏥', 'place': 'Dallas, TX'}]

countryHappinness = {}
#for tweet in testData:
for tweet in tweets:
  if ',' in tweet['place']:
    code = tweet['place'].split(', ')[1]
    for value in states:
      if value['state'] == code:
        happinessIndex = 0
        for word in tweet['text'].split():
          if word.lower() in affinn:
            happinessIndex += int(affinn[word.lower()])
        if value['state'] in countryHappinness:
          totalHappiness = countryHappinness[value['state']]['happiness'] + happinessIndex
        else:
          totalHappiness = happinessIndex
        countryHappinness.update({ value['state']: { 'happiness': totalHappiness, 'latitude': value['latitude'], 'longitude': value['longitude'], 'name': value['name'] } })
print(countryHappinness)

{'CA': {'happiness': 12, 'latitude': '36.778261', 'longitude': '-119.417932', 'name': 'California'}, 'AL': {'happiness': 4, 'latitude': '32.318231', 'longitude': '-86.902298', 'name': 'Alabama'}, 'TN': {'happiness': 1, 'latitude': '35.517491', 'longitude': '-86.580447', 'name': 'Tennessee'}, 'TX': {'happiness': 9, 'latitude': '31.968599', 'longitude': '-99.901813', 'name': 'Texas'}, 'MD': {'happiness': -5, 'latitude': '39.045755', 'longitude': '-76.641271', 'name': 'Maryland'}, 'MI': {'happiness': 0, 'latitude': '44.314844', 'longitude': '-85.602364', 'name': 'Michigan'}, 'FL': {'happiness': 9, 'latitude': '27.664827', 'longitude': '-81.515754', 'name': 'Florida'}, 'MN': {'happiness': 6, 'latitude': '46.729553', 'longitude': '-94.6859', 'name': 'Minnesota'}, 'NC': {'happiness': 6, 'latitude': '35.759573', 'longitude': '-79.0193', 'name': 'North Carolina'}, 'PA': {'happiness': -4, 'latitude': '41.203322', 'longitude': '-77.194525', 'name': 'Pennsylvania'}, 'NY': {'happiness': 0, 'latitu

#### Map
A continuación se muestra un mapa de Estados Unidos con sus estados para comprobar que los Tweets se situan correctamente, así como el análisis de sentimiento en cada estado

In [8]:
!pip install rasterio
!pip install earthpy

     |████████████████████████████████| 19.1MB 1.4MB/s 
     |████████████████████████████████| 1.4MB 5.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.0MB 45.9MB/s 
     |████████████████████████████████| 6.5MB 38.8MB/s 
     |████████████████████████████████| 14.8MB 313kB/s 
  Created wheel for earthpy: filename=earthpy-0.9.2-cp37-none-any.whl size=1365089 sha256=2d97a983368b1c04154cccbc8e776a436e40d6b7404541be8c7fc85c30ddd69d
  Stored in directory: /root/.cache/pip/wheels/27/f9/a5/026bb95ef7b12f9343b842da6897b4001c5b902550095cbbbe
Successfully built earthpy


In [9]:
import os 
import folium
from folium import plugins
import rasterio as rio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import earthpy as et

In [ ]:
# Crear el mapa con centro en USA
m = folium.Map(location=[37.09024,	-95.712891], zoom_start=4)

# Añadir Markers al mapa
for country in countryHappinness:
  color = 'black'
  happinessLevel = countryHappinness[country]['happiness']
  if happinessLevel < -2:
    color = 'red'
  elif happinessLevel >= -2 and happinessLevel <= 2:
      color = '#BDB76B'
  elif happinessLevel > 2:
      color = 'green'
  folium.Marker(
    location=[float(countryHappinness[country]['latitude']), float(countryHappinness[country]['longitude'])],
    icon=folium.DivIcon(html=f"""<div style="color: {color}; font-size: 14px">{countryHappinness[country]['happiness']}</div>""")
  ).add_to(m)

# Mostrar mapa
m

## CONCLUSIÓN: 5 ESTADOS MÁS FELICES



In [ ]:
import operator

result = []
for s in sorted(countryHappinness.items(), key=lambda k_v: k_v[1]['happiness']):
  result.append(s)
result[len(result) - 5:len(result)]

[('TX',
  {'happiness': 9,
   'latitude': '31.968599',
   'longitude': '-99.901813',
   'name': 'Texas'}),
 ('FL',
  {'happiness': 9,
   'latitude': '27.664827',
   'longitude': '-81.515754',
   'name': 'Florida'}),
 ('OR',
  {'happiness': 11,
   'latitude': '43.804133',
   'longitude': '-120.554201',
   'name': 'Oregon'}),
 ('CA',
  {'happiness': 12,
   'latitude': '36.778261',
   'longitude': '-119.417932',
   'name': 'California'}),
 ('KS',
  {'happiness': 13,
   'latitude': '39.011902',
   'longitude': '-98.484246',
   'name': 'Kansas'})]

#PARTE II:
Realizar un mapa de España que permita ver dónde se tuitea y en qué idioma se tuitea

####Obtener Tweets
En esta ocasión he cambiado las coordenadas, pero el patrón para crear el streaming y modificar el tiempo de duración es el mismo

In [ ]:
from tweepy.streaming import StreamListener
from tweepy import Stream
import json
import time

class StdOutListener(StreamListener):
    def on_data(self, data):
      tweet = json.loads(data)
      tweets.append({ 'text': tweet['text'], 'place': tweet['place']['full_name'], 'lang': tweet['lang'], 'coordinates': tweet['coordinates'] })
      return True

    def on_error(self, status):
        print(status)

tweets = []
l = StdOutListener()
stream = Stream(auth, l)
stream.filter(locations=[-9.39288367353, 35.946850084, 3.03948408368, 43.7483377142], is_async=True)
time.sleep(10)
stream.disconnect()

# DEBUGGING del resultado
#tweets

In [35]:
# Igual que antes, he utilizado datos fijos para incrementar la velocidad de 
#desarrollo y evitar cargar la API de Tweitter
test_data = [{'coordinates': None,
  'lang': 'en',
  'place': 'Barcelona, Spain',
  'text': '@GameChangerFit1 The fact that you go straight to opening a door and being polite as what women are looking for as… https://t.co/ZABzOHzFfn'},
 {'coordinates': None,
  'lang': 'es',
  'place': 'Madrid, España',
  'text': '@Musincronizados @radio_consuegra @paulaferraz97 @Contraclub @enclavedeindie @revistaindie @AbsoluteAMusic… https://t.co/2tW4qVKvw1'},
 {'coordinates': None,
  'lang': 'es',
  'place': 'Granada, España',
  'text': 'Un poquito intensa en mi balcon https://t.co/nMsXcfe9Kp'},
 {'coordinates': None,
  'lang': 'es',
  'place': 'Málaga, España',
  'text': 'beleeen tiiaaaa con lo que me morí d ganas d verte🥺 pero aguantaremos el tiron💪🏻💖'},
 {'coordinates': None,
  'lang': 'es',
  'place': 'Cártama, España',
  'text': '@tracygerman1 @carlosmiquelf1 @AlpineF1Team @alo_oficial @ScuderiaFerrari @Carlossainz55 Pues a mi me gusta más el… https://t.co/hEnzQ6d1J5'},
 {'coordinates': {'coordinates': [-6.0052, 37.38358056], 'type': 'Point'},
  'lang': 'es',
  'place': 'Sevilla, España',
  'text': 'Acaba de publicar una foto en Triana (Sevilla) https://t.co/t2fzv7CKv8'},
 {'coordinates': None,
  'lang': 'es',
  'place': 'Gijón, España',
  'text': 'bueno mira lo voy a subir pq estoy súper emocionada 😭😭😭😭\n4 feb (primera foto)\n10 marzo (segunda y tercera) https://t.co/GTsce6mZ3y'},
 {'coordinates': None,
  'lang': 'ca',
  'place': 'Barcelona, Espanya',
  'text': '@biel_ricard Una vergonya.'},
 {'coordinates': None,
  'lang': 'und',
  'place': 'Alicante, España',
  'text': '@wannabedriver_ 😂😂😂'},
 {'coordinates': None,
  'lang': 'es',
  'place': 'Rota, España',
  'text': '@IdiazAyuso\nSsshhhhhh... #IDA.\nSOCIALISMO Y LIBERTAD.'},
 {'coordinates': None,
  'lang': 'en',
  'place': 'Gibraltar',
  'text': '@synpat_crypto Wow! I would have hoped for better than that.\nI really struggle to elicit reasoned responses for why… https://t.co/qidfELFbQK'},
 {'coordinates': None,
  'lang': 'cy',
  'place': 'Sevilla, España',
  'text': 'Muniain 😍'}]

####Obtener las provincias de España y sus coordenadas a partir de un json
En esta ocasión en lugar de subir manualmente el fichero he decidido probar la funcionalidad de upload y utilizar un json que contiene las provincias de España con sus coordenadas. El fichero se llama spain.json y puede encontrar en los archivos adjuntos en el zip

In [ ]:
import json
from google.colab import files

uploaded = files.upload()
data = next(iter(uploaded.values()))
spains = json.loads(data.decode())

# DEBUGGING del resultado
#spains

In [40]:
languagesPerCity = {}
for tweet in test_data:
  where = tweet['place'].split(', ')[0]
  for placeOfSpain in spains:
    if where == placeOfSpain['city']:
      if where in languagesPerCity:
        languagesPerCity[where]['languages'].append(tweet['lang'])
      else:
        langList = [tweet['lang']]
        languagesPerCity.update({ where: { 'languages': langList, 'lat': placeOfSpain['lat'], 'lon': placeOfSpain['lng'] } })
# DEBUGGING del resultado
languagesPerCity    

{'Alicante': {'languages': ['und'], 'lat': '38.3453', 'lon': '-0.4831'},
 'Barcelona': {'languages': ['en', 'ca'], 'lat': '41.3825', 'lon': '2.1769'},
 'Gijón': {'languages': ['es'], 'lat': '43.5293', 'lon': '-5.6773'},
 'Granada': {'languages': ['es'], 'lat': '37.1781', 'lon': '-3.6008'},
 'Madrid': {'languages': ['es'], 'lat': '40.4189', 'lon': '-3.6919'},
 'Málaga': {'languages': ['es'], 'lat': '36.7167', 'lon': '-4.4167'},
 'Sevilla': {'languages': ['es', 'cy'], 'lat': '37.3828', 'lon': '-5.9731'}}

##CONCLUSIÓN: LUGARES DE ESPAÑA DÓNDE SE "TUITEA" Y EN QUE IDIOMA
A continuación muestro el mapa con markers en cada provincia, para ver los idiomas hay que hacer click en el marker. He dejado el valor "und" ya que significa indeterminado y no he querido aplicar ningún sesgo a los datos tratados, no obstante si se desea eliminar se puede aplicar un slice o bien filtrar los tweets para que solo se utilicen aquellos que la property `lang` sea distinta de `und`

In [48]:
# Crear el mapa con centro en España
m = folium.Map(location=[40.463667, -3.74922], zoom_start=6)

# Añadir Markers al mapa
for city in languagesPerCity:
  popup = '<h6>Idiomas de los Tweets: </h6>';
  for lang in languagesPerCity[city]['languages']:
    popup += '<span>'+lang+'</span><br>'
  folium.Marker(
    location=[float(languagesPerCity[city]['lat']), float(languagesPerCity[city]['lon'])],
    icon=folium.Icon(color="red", icon="info-sign"),
    popup=popup
  ).add_to(m)

# Mostrar mapa
m